In [2]:
import numpy as np
import matplotlib.pyplot as plt

from Code_Ring_Network import CodeRingNetwork

In [13]:
# Specificity Development
w_mc = np.random.rand(16,12)
plt.matshow(w_mc)
plt.colorbar()
means = np.ndarray(w_mc.shape)
for i in range(w_mc.shape[1]):
    j = (i+1) % w_mc.shape[1]
    means[:,i] = (w_mc[:, i] + w_mc[:, j]) / 2
w_mc = np.repeat(w_mc, 2, axis=1)

w_mc[:, 1::2] = means
plt.matshow(w_mc)
plt.colorbar()

d = int(np.sqrt(w_mc.shape[0]))
w_mc = w_mc.reshape((d, d, w_mc.shape[-1]))
w_mc = np.repeat(w_mc, 2, axis=0)
w_mc = np.repeat(w_mc, 2, axis=1)
w_mc = w_mc + ((np.random.rand(*w_mc.shape) * 0.1) - 0.05)
plt.matshow(w_mc[:,:,0])
plt.colorbar()
plt.matshow(w_mc.reshape(-1, w_mc.shape[-1]))
plt.colorbar()
# plt.matshow(w_mc.reshape(144*4,36*2).T + ((np.random.rand(144*4,36*2).T * 0.1) - 0.1))

In [14]:
# Redundancy Development
w_mc = np.random.rand(16,12)
plt.matshow(w_mc)
plt.colorbar()

w_mc = np.tile(w_mc, 2)
plt.matshow(w_mc)
plt.colorbar()

c = int(w_mc.shape[1] // 2)
w_red = np.ndarray((w_mc.shape[0], c, 2))
w_red[:,:,0] = w_mc[:, :c]
w_red[:,:,1] = w_mc[:, c:]

plt.matshow(w_red[:,:,0])
plt.colorbar()

plt.matshow(w_red[:,:,1])
plt.colorbar()

In [15]:
from scipy.stats import kendalltau
import numpy as np
import matplotlib.pyplot as plt

from utilities import exponential

In [16]:
def code_spread_metric(code_activity, **metric_kwargs):
    n = 36
    if code_activity.shape == (n,1):
        code_activity = code_activity.flatten()
    num_high = np.count_nonzero(code_activity > 0.5)
    high_idxs = np.argwhere(code_activity > 0.5).flatten()
    spreads = np.zeros((n, n, 2)) + 9999
    for i in high_idxs:
        for j in high_idxs:
            if num_high == np.count_nonzero(code_activity[i:j+1] > 0.5):
                spreads[i][j][0] = np.abs(i - j) + 1
            if num_high == np.count_nonzero(np.concatenate((code_activity[0:i+1], code_activity[j:n])) > 0.5):
                spreads[i][j][1] = n - np.abs(i - j) + 1
    min_spread = np.min(spreads)
    top = exponential(min_spread, rate=-0.05, center=num_high, init_val=1)
    bottom = 1 + (0.2  * np.abs(num_high - (1*n//4)))
    return top / bottom

In [17]:
n = 36
ideal = np.linspace(0.5, 1, int(n // 4))

# bimodal noise, so 3/4 of values are low, 1/4 are high
noise_rate = 8
noise1 = np.random.exponential(1 / noise_rate, 3*n//4)
noise2 = 1 - np.random.exponential(1 / noise_rate, n//4)
noise = np.concatenate((noise1, noise2))
np.random.shuffle(noise)
# code input is just noise during pretraining
c = np.clip(noise, 0, 1)

spread_score = code_spread_metric(c)

top_idxs = np.sort(np.argsort(c)[-n//4:])
kendall = np.abs(kendalltau(c[top_idxs], ideal).statistic)

print(c)
print(c[top_idxs])

print(f'Spread score: {spread_score}')

print(f'Kendall: {kendall}')

print(f'Total score: {(theta * spread_score) + (kendall * (1 - theta))}')

[0.01591844 0.92145515 0.1400118  0.18791357 0.07939174 0.81903644
 0.04063173 0.27006131 0.98682106 0.93233561 0.05027459 0.69968791
 0.01395029 0.00998255 0.02313115 0.14665214 0.03831649 0.09364141
 0.12729851 0.00370812 0.08631523 0.12612955 0.2464463  0.83956042
 0.31660223 0.01787498 0.08409228 0.9576185  0.9112309  0.21865069
 0.17013346 0.04476276 0.02441873 0.87774952 0.99242519 0.24079435]
[0.92145515 0.81903644 0.98682106 0.93233561 0.83956042 0.9576185
 0.9112309  0.87774952 0.99242519]
Spread score: 0.3936387939508456
Kendall: 0.16666666666666666


NameError: name 'theta' is not defined

In [17]:
import matplotlib
matplotlib.use('TkAgg')
crn = CodeRingNetwork(36,36,1,36,8,8,0,1,0.02)
map_size = int(crn.map_layer.d1*crn.map_layer.d2)
durations=0.2
t_max=70
t_steps=700
f,axs = plt.subplots(crn.map_layer.d1, crn.map_layer.d2, figsize=(16,8))
for i in range(map_size):
    (r,c) = crn.map_layer.convert_to_coord(i)

    noise_rate = 8
    noise1 = np.random.exponential(1 / noise_rate, 3*crn.code_layer.num_code_units//4)
    noise2 = 1 - np.random.exponential(1 / noise_rate, crn.code_layer.num_code_units//4)
    noise = np.concatenate((noise1, noise2))
    # np.random.shuffle(noise)

    code_output = np.clip(noise, 0, 1).reshape(crn.code_layer.num_code_units, 1)
    code_output = code_output

    # determine output of code layer (input into ring layer)
    ring_input = crn.code_layer.weights_to_ring_from_code @ code_output

    # determine activity of duration layer
    # TODO: right now, this is just a constant
    dur_output = crn.duration_layer.activate(durations)

    # integrate ring layer model over time
    v_series, z_series, u_series, I_prime_series = crn.ring_layer.activate(ring_input, dur_output, t_max=t_max, t_steps=t_steps)
    
    # apply model results to the drawing system
    x_series, y_series = crn.ring_layer.create_drawing(z_series, t_steps)

    # evaluate drawing
    # score, curvatures, intersec_pts, intersec_times = crn.evaluate(x_series, y_series, t_steps=t_steps)
    score = crn.eval_kendall_spread(code_activity=code_output, theta=0.8)
    # print(score)
                                    
    # generate drawing for current neuron
    crn.plot_final_doodle(ax=axs[r][c], xs=x_series, ys=y_series, intersec_pts=np.array([]), individualize_plot=False)
    axs[r][c].set_xlim([-50,50])
    axs[r][c].set_ylim([-50,50])
    axs[r][c].set_xlabel(f'{np.round(score,3)}')
    axs[r][c].set_box_aspect(1)

plt.tight_layout()
plt.show()

ID string: 2024-03-03 164233434479
